# NetCDF zu GeoTIFF Konvertierung
Dieses Skript lädt NetCDF-Dateien, extrahiert relevante Daten, und speichert sie als GeoTIFF-Dateien. Da NetCDF-Dateien oft Zeitschritte enthalten, wird das Skript die Daten über die Zeitachsen iterieren und mehrere TIFF-Dateien erzeugen (eines pro Zeitschritt).

### Bibliotheken:
- `xarray`: Für das Laden und Bearbeiten von NetCDF-Daten.
- `rasterio`: Für die Erstellung von GeoTIFF-Dateien.
- `numpy`: Für die Arbeit mit numerischen Daten.

### Vorgehen:
1. Bibliotheken laden und NetCDF-Datei öffnen.
2. Überprüfen der Datenstruktur (Dimensionen, Variablen).
3. Konvertierung der NetCDF-Daten (pro Zeitschritt) in GeoTIFF.


In [1]:
# Bibliotheken importieren
import xarray as xr
import rasterio
import numpy as np
import os


## Schritt 1: NetCDF-Datei laden
Verwende die Bibliothek `xarray`, um die NetCDF-Datei zu öffnen und einen Überblick über die enthaltenen Daten und Dimensionen zu bekommen.
Bitte passe den Pfad zu deiner NetCDF-Datei an.


In [ ]:
# NetCDF-Datei laden (Pfad zur Datei anpassen)
nc_file_path = 'dein_pfad/zur_netCDF_datei.nc'

# Öffnen der NetCDF-Datei mit xarray
ds = xr.open_dataset(nc_file_path)

# Überblick über die Datenstruktur (Dimensionen, Variablen)
print(ds)

# Wenn du die Zeitschritte oder andere Variablen sehen möchtest:
print("Zeitliche Dimension:", ds.coords['time'].values)


## Schritt 2: Auswahl der Variable und des Zeitintervalls
Wähle die Variable aus der NetCDF-Datei, die du konvertieren möchtest (z.B. Temperatur, Niederschlag, etc.), und definiere den Zeitschritt, den du in eine GeoTIFF-Datei umwandeln möchtest.


In [ ]:
# Wähle die Variable, die konvertiert werden soll (Name anpassen)
variable_name = 'temperature'  # Beispiel: 'temperature', 'precipitation', etc.

# Greife auf die Daten der Variable zu
data_var = ds[variable_name]

# Zeitschritt auswählen (Index anpassen)
# Hier wähle den ersten Zeitschritt als Beispiel
time_index = 0

# Daten des ausgewählten Zeitschritts extrahieren
data_at_time = data_var.isel(time=time_index)

# Überprüfen der extrahierten Daten
print(data_at_time)


## Schritt 3: Konvertierung in GeoTIFF
Nun konvertieren wir die extrahierten Rasterdaten in eine GeoTIFF-Datei. Dafür wird die Bibliothek `rasterio` verwendet. Wir müssen auch die Geoinformationen wie die Projektion (z.B. WGS84) und die Bounding Box bereitstellen.


In [ ]:
# Erstellen eines GeoTIFF-Dateipfads
output_tif = 'output_raster.tif'

# Beispielhafte affine Transformation und CRS (proj4 oder EPSG-Code)
transform = rasterio.transform.from_origin(west=0, north=0, xsize=0.1, ysize=0.1)  # Beispielhafte Parameter
crs = 'EPSG:4326'  # WGS84

# Öffnen einer neuen TIFF-Datei zum Schreiben
with rasterio.open(
    output_tif, 
    'w', 
    driver='GTiff', 
    height=data_at_time.shape[0], 
    width=data_at_time.shape[1], 
    count=1, 
    dtype=str(data_at_time.dtype), 
    crs=crs, 
    transform=transform
) as dst:
    # Schreibe die Daten in die TIFF-Datei
    dst.write(data_at_time.values, 1)

print(f'Datei {output_tif} erfolgreich erstellt.')


## Schritt 4: Konvertierung für alle Zeitschritte (optional)
Falls die NetCDF-Datei mehrere Zeitschritte enthält und du alle in separate TIFF-Dateien konvertieren möchtest, kannst du über die Zeitachse iterieren und für jeden Zeitschritt eine TIFF-Datei erzeugen.


In [ ]:
# Erstellen eines Ordners zum Speichern der GeoTIFFs
output_folder = 'output_tiffs'
os.makedirs(output_folder, exist_ok=True)

# Über die Zeitachsen iterieren
for i in range(len(ds.coords['time'])):
    # Extrahiere Daten für den aktuellen Zeitschritt
    data_at_time = data_var.isel(time=i)
    
    # Dateiname für den aktuellen Zeitschritt
    output_tif = os.path.join(output_folder, f'raster_time_{i}.tif')
    
    # GeoTIFF-Datei erstellen
    with rasterio.open(
        output_tif, 
        'w', 
        driver='GTiff', 
        height=data_at_time.shape[0], 
        width=data_at_time.shape[1], 
        count=1, 
        dtype=str(data_at_time.dtype), 
        crs=crs, 
        transform=transform
    ) as dst:
        dst.write(data_at_time.values, 1)

    print(f'Datei {output_tif} für Zeitschritt {i} erfolgreich erstellt.')


## Fazit
Das Skript ermöglicht die Konvertierung von NetCDF-Daten in GeoTIFF-Dateien, wobei Zeitschritte berücksichtigt werden. Diese Methode kann verwendet werden, um rasterbasierte Zeitreihenanalysen in einem georeferenzierten Format (TIFF) zu speichern und weiterzuverarbeiten.
